#SNP Detection

##Setting the dependencies

In [5]:
snpFile   = '/home/senne/nanopore/SNP/known_SNP_sequence/SNP_sequence.fasta'  
readFile  = '/home/senne/nanopore/SNP/Nanopore_data/2_potential_snp_amplicons_3mism.fasta'
resultDir = '/home/senne/nanopore/SNP/results_yannick'
fastq_file_name= '/home/senne/nanopore/SNP/Nanopore_data/ligatedSNPs.fastq'
bwa       = '/opt/tools/bwa-0.7.15'     # v0.7.5
samtools  = '/opt/tools/samtools-1.3.1' # v1.3.1
bcftools  = '/opt/tools/bcftools-1.3.1' # v1.3.1

# Check
!ls {snpFile}
print('Number of SNPs in reference file:')
!grep -c ">" {snpFile}


/home/senne/nanopore/SNP/known_SNP_sequence/SNP_sequence.fasta
Number of SNPs in reference file:
52


## Info on the fastq file

In [6]:
#from collections import Counter
#fastq_file_name = 'ligatedSTRs1.fastq'
#histogram_data = Counter()
hist_array = []
hist_arrayG = []
hist_arrayA = []
hist_arrayC = []
hist_arrayT = []
with open(fastq_file_name, "rb") as infile:
    for line in infile:
        if line.startswith(b'G'):
            read_length = len(line[:-1]) #Last char is \n
            #histogram_data[read_length] += 1
            hist_arrayG.append(read_length)
with open(fastq_file_name, "rb") as infile:
    for line in infile:
        if line.startswith(b'A'):
            read_length = len(line[:-1]) #Last char is \n
            #histogram_data[read_length] += 1
            hist_arrayA.append(read_length) 
with open(fastq_file_name, "rb") as infile:
    for line in infile:
        if line.startswith(b'C'):
            read_length = len(line[:-1]) #Last char is \n
            #histogram_data[read_length] += 1
            hist_arrayC.append(read_length) 
with open(fastq_file_name, "rb") as infile:
    for line in infile:
        if line.startswith(b'T'):
            read_length = len(line[:-1]) #Last char is \n
            #histogram_data[read_length] += 1
            hist_arrayT.append(read_length)

hist_array.extend(hist_arrayA)
hist_array.extend(hist_arrayC)
hist_array.extend(hist_arrayG)
hist_array.extend(hist_arrayT)

import numpy as np

len (hist_array), np.mean(hist_array), np.median(hist_array) , max(hist_array), np.std(hist_array)

(14324, 507.74413571628037, 411.0, 5850, 324.5996162457601)

##Map reads to reference sequences

In [7]:
# Map reads to reference sequences

# Build index of the references
!{bwa} index {snpFile}

# Map reads
!{bwa} mem -t 10 -k 14 -W 20 -r 10 -O 6,6 -E 1 {snpFile} {readFile} > {resultDir}/ONTGAP4ttt.sam

# Make sorted bam and index

!{samtools} view -Sbu {resultDir}/ONTGAP4ttt.sam | {samtools} sort -o {resultDir}/ONTGAP4ttt_sorted.bam -
!{samtools} index {resultDir}/ONTGAP4ttt_sorted.bam {resultDir}/ONTGAP4ttt_sorted.bam.bai
print('Done')



[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.15-r1140
[main] CMD: /opt/tools/bwa-0.7.15 index /home/senne/nanopore/SNP/known_SNP_sequence/SNP_sequence.fasta
[main] Real time: 0.053 sec; CPU: 0.015 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 23057 sequences (1988758 bp)...
[M::mem_process_seqs] Processed 23057 reads in 0.983 CPU sec, 0.102 real sec
[main] Version: 0.7.15-r1140
[main] CMD: /opt/tools/bwa-0.7.15 mem -t 10 -k 14 -W 20 -r 10 -O 6,6 -E 1 /home/senne/nanopore/SNP/known_SNP_sequence/SNP_sequence.fasta /home/senne/nanopore/SNP/Nanopore_data/2_potential_snp_amplicons_3mism.fasta
[main] Real time: 0.282 sec; CPU: 1.017 sec
Done


In [8]:
# Display some stats

!{samtools} flagstat {resultDir}/ONTGAP4ttt_sorted.bam


23062 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
5 + 0 supplementary
0 + 0 duplicates
8034 + 0 mapped (34.84% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


## Generate vcf file

In [9]:
# Generate vcf file from bam file. Needs the reference and its index file 

# Note: the commands below are for samtools and bcftools v1.3.1 and will not work on v0.1.19!

# Reporting all positions
!{samtools} mpileup -d 100000 -uf {snpFile} {resultDir}/ONTGAP4ttt_sorted.bam | {bcftools} call -V indels -m - > {resultDir}/ONTGAP4ttt_sorted.bam.vcf

# Reporting variants only (excludes SNPs homozygous for reference allele)
!{samtools} mpileup -d 100000 -uf {snpFile} {resultDir}/ONTGAP4ttt_sorted.bam | {bcftools} call -V indels -mv - > {resultDir}/ONTGAP4yyy_sorted.bam.vcf

print('Done')

[mpileup] 1 samples in 1 input files
Note: Neither --ploidy nor --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
Note: Neither --ploidy nor --ploidy-file given, assuming all sites are diploid
Done


##Generate the SNP profile 

In [10]:
# Get SNP profile


snpData = {}

with open(resultDir+'/ONTGAP4ttt_sorted.bam.vcf') as f:
    for l in f:
        if l.startswith('#'):
            continue
            
        snp, pos, id, ref, alt, qual, filter, info, d, dd = l.split()
        
        # Our SNP of interest is always at position 26 of the reference
        if int(pos) != 26:
            continue

        par = {}
        for p in info.split(';'):
            pv = p.split('=')
            par[pv[0]] = pv[1]
        
        snpData[snp] = {'pos': pos, 'ref': ref, 'alt': alt, 'qual': qual, 'filter': filter, 'info': par}

# DEBUG
print('Got data for {} SNPs:'.format(len(snpData)))

# Save/print results
with open(resultDir + '/ttt_profile.csv', 'w') as f:
    # Table header
    f.write('snp, coverage, ref_allele, ref_percent, alt_allele, alt_percent, genotype\n')
    
    # Table data
    for s in sorted(snpData.keys()):
        totalDepth = int(snpData[s]['info']['DP'])
        depthList  = [int(d) for d in snpData[s]['info']['DP4'].split(',')]
        refDepth   = sum(depthList[0:2])
        altDepth   = sum(depthList[2:4])
        
        # Estimate the diploid genotype: when the minor allele is more than 10 times weaker than the major allele,
        # we should ignore it for a pure sample?
        if refDepth > altDepth and altDepth/refDepth < 0.1:
            genotype = snpData[s]['ref'] + snpData[s]['ref']
        elif altDepth > refDepth and refDepth/altDepth < 0.1:
            genotype = snpData[s]['alt'] + snpData[s]['alt']
        else:
            genotype = snpData[s]['ref'] + snpData[s]['alt']
        
        if snpData[s]['alt'] == '.':
            # Only 1 allele was observed
            f.write(','.join([s, str(totalDepth), snpData[s]['ref'], '{:.1f}'.format(100*refDepth/totalDepth), '', '', snpData[s]['ref']+snpData[s]['ref']]) + '\n')
            # DEBUG
            print('  {} ({})  {} ({:.1f} %)'.format(s, totalDepth, snpData[s]['ref'], 100*refDepth/totalDepth))
        else:
            # Two alleles were observed
            f.write(','.join([s, str(totalDepth), snpData[s]['ref'], '{:.1f}'.format(100*refDepth/totalDepth), snpData[s]['alt'], '{:.1f}'.format(100*altDepth/totalDepth), genotype]) + '\n')
            # DEBUG
            print('  {} ({})  {} ({:.1f} %)  {} ({:.1f} %)'.format(s, totalDepth, snpData[s]['ref'], 100*refDepth/totalDepth, snpData[s]['alt'], 100*altDepth/totalDepth))


Got data for 52 SNPs:
  rs1005533 (99)  A (61.6 %)  G (38.4 %)
  rs1015250 (67)  C (3.0 %)  G (97.0 %)
  rs1024116 (111)  A (38.7 %)  G (61.3 %)
  rs1028528 (136)  A (2.9 %)  G (97.1 %)
  rs1029047 (24)  A (100.0 %)
  rs1031825 (82)  A (23.2 %)  C (76.8 %)
  rs10495407 (188)  A (60.6 %)  G (39.4 %)
  rs1335873 (120)  A (1.7 %)  T (98.3 %)
  rs1355366 (166)  A (5.4 %)  G (94.6 %)
  rs1357617 (220)  A (100.0 %)
  rs1360288 (184)  C (59.8 %)  T (40.2 %)
  rs1382387 (207)  G (2.4 %)  T (97.6 %)
  rs1413212 (168)  A (64.9 %)  G (35.1 %)
  rs1454361 (200)  A (100.0 %)
  rs1463729 (157)  A (97.5 %)
  rs1490413 (235)  A (98.3 %)
  rs1493232 (78)  A (67.9 %)  C (32.1 %)
  rs1528460 (85)  C (2.4 %)  T (97.6 %)
  rs1886510 (144)  C (4.9 %)  T (95.1 %)
  rs1979255 (301)  C (98.0 %)
  rs2016276 (70)  A (95.7 %)
  rs2040411 (72)  A (0.0 %)  G (100.0 %)
  rs2046361 (111)  A (3.6 %)  T (96.4 %)
  rs2056277 (199)  C (47.7 %)  T (52.3 %)
  rs2076848 (127)  A (72.4 %)  T (27.6 %)
  rs2107612 (49)  A (0.0